In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import seaborn as sns

import datetime
pd.set_option('display.max_columns', None)
#########################################################
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
ss = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')


Time Series Analysis
The goal of time series analysis is to find patterns in the data and use the data for predictions. For example, if your data is affected by past data, one way to model that behavior is through the AR process.

lets visualize data as whole..


In [ ]:
import plotly.express as px
px.defaults.color_continuous_scale = px.colors.sequential.Blackbody
fig = px.colors.qualitative.swatches()
fig1 = px.line(train, x='date_time', y="target_carbon_monoxide")
fig1.update_traces(line_color='#456987')
fig2 = px.line(train, x='date_time', y="target_nitrogen_oxides")
fig3 = px.line(train, x='date_time', y="target_benzene")
fig3.update_traces(line_color='red')
fig1.show()
fig2.show()
fig3.show()

DEC 2010 this is the month where concentration levels are highers as different months 

question- what could be possible reassons? anwers in commentss

how is the distribution of CO,benzene and NO day wise ?

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv', parse_dates=['date_time'],index_col= ['date_time'],)

perday_data=df["2010-04-10" : "2010-04-10"]

perday_data=perday_data.drop(columns=['deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5'])

perday_data=perday_data.reset_index()

fig1 = px.line(perday_data, x="date_time", y='target_carbon_monoxide',
              hover_data={"date_time": "|%B %d, %Y"},
              title='perday levels of carbon_monoxide')
fig1.update_xaxes(tickformat="%H:%M h")

fig2 = px.line(perday_data, x="date_time", y='target_benzene',
              hover_data={"date_time": "|%B %d, %Y"},
              title='perday levels of benzene')
fig2.update_xaxes(tickformat="%H:%M h")

fig3 = px.line(perday_data, x="date_time", y='target_nitrogen_oxides',
              hover_data={"date_time": "|%B %d, %Y"},
              title='perday levels of nitrogen_oxides')
fig3.update_xaxes(tickformat="%H:%M h")

fig1.show()
fig2.show()
fig3.show()



feature engineering notebooks are dominated by 'working_hour'   :) 

7:00 to 11:00 is office opening time there is a peak?  most of the peoples rushing toward their office

guess there is one more rush in the evening while returning home. 

#non-bussiness hours  

In [ ]:
import plotly.figure_factory as ff

hist_data = [train['target_carbon_monoxide']]

group_labels = ['CO']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels,bin_size=.1)
fig.show()



In [ ]:
hist_data = [train['target_nitrogen_oxides']]

group_labels = ['NO']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels)
fig.show()


In [ ]:
hist_data = [train['target_benzene']]

group_labels = ['C6H6']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels)
fig.show()


there are outliers in the data set specially in nitrogen oxides, i guess they are from month December.

In [ ]:
import seaborn as sns
ax = sns.boxplot(y="target_nitrogen_oxides", data=train)

visualizing statistical relationship

In [ ]:
sns.relplot(x="target_benzene", y="target_carbon_monoxide", data=train )
sns.relplot(x="target_benzene", y="target_nitrogen_oxides", data=train )
sns.relplot(x="target_carbon_monoxide", y="target_nitrogen_oxides", data=train)



there is a stastical relationship between CO,NO and benzene. could be used as feature in predicting value of other, as chemically benzene and carbon monoxide both contains carbon. 

carbon monoxide and nitrogen oxides are major contributer from road traffic.

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

sns.kdeplot(ax=axes[0,0],data=train, x="target_carbon_monoxide", y="sensor_1", fill=True)
sns.kdeplot(ax=axes[0,1],data=train, x="target_carbon_monoxide", y="sensor_2", fill=True)
sns.kdeplot(ax=axes[0,2],data=train, x="target_carbon_monoxide", y="sensor_3", fill=True)
sns.kdeplot(ax=axes[1,0],data=train, x="target_carbon_monoxide", y="sensor_4", fill=True)
sns.kdeplot(ax=axes[1,1],data=train, x="target_carbon_monoxide", y="sensor_5", fill=True)